In [1]:
from ome_zarr_models.v04 import Image
from ome_zarr_models import open_ome_zarr
from pathlib import Path
import zarr

path = Path("/media/alessandro/T7/tibiae/pyramidal_chunked_tibia_1")
assert path.exists()
zarr_group = zarr.open(path, mode="r+")
ome_zarr_group = open_ome_zarr(zarr_group)
print(f"Type of the dataset: {type(ome_zarr_group)}")
print(f"OME-Zarr version: {ome_zarr_group.ome_zarr_version}")

Type of the dataset: <class 'ome_zarr_models.v04.image.Image'>
OME-Zarr version: 0.4


In [2]:
ome_zarr_image = Image.from_zarr(zarr_group)

print(ome_zarr_image.attributes)
metadata = ome_zarr_image.attributes
from rich.pretty import pprint
pprint(metadata)
pprint(metadata.multiscales[0].datasets)

zarr_arr = zarr_group[metadata.multiscales[0].datasets[0].path]
pprint(zarr_arr)



multiscales=[Multiscale(axes=[Axis(name='x', type='space', unit='um'), Axis(name='y', type='space', unit='um'), Axis(name='z', type='space', unit='um')], datasets=(Dataset(path='level0', coordinateTransformations=(VectorScale(type='scale', scale=[1.0, 1.0, 1.0]), VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0]))), Dataset(path='level1', coordinateTransformations=(VectorScale(type='scale', scale=[2.0, 2.0, 2.0]), VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0]))), Dataset(path='level2', coordinateTransformations=(VectorScale(type='scale', scale=[4.0, 4.0, 4.0]), VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])))), coordinateTransformations=(VectorScale(type='scale', scale=[10.0, 10.0, 10.0]),), metadata=None, name='mouse tibia', type=None, version='0.4')] omero=None


ImageAttrs(
│   multiscales=[
│   │   Multiscale(
│   │   │   axes=[
│   │   │   │   Axis(name='x', type='space', unit='um'),
│   │   │   │   Axis(name='y', type='space', unit='um'),
│   │   │   │   Axis(name='z', type='space', unit='um')
│   │   │   ],
│   │   │   datasets=(
│   │   │   │   Dataset(
│   │   │   │   │   path='level0',
│   │   │   │   │   coordinateTransformations=(
│   │   │   │   │   │   VectorScale(type='scale', scale=[1.0, 1.0, 1.0]),
│   │   │   │   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   │   │   │   )
│   │   │   │   ),
│   │   │   │   Dataset(
│   │   │   │   │   path='level1',
│   │   │   │   │   coordinateTransformations=(
│   │   │   │   │   │   VectorScale(type='scale', scale=[2.0, 2.0, 2.0]),
│   │   │   │   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   │   │   │   )
│   │   │   │   ),
│   │   │   │   Dataset(
│   │   │   │   │   path='level2',
│   │   │   │   │   coordinateTransformations=(
│   │   │   │   │   │   VectorScale(type='scale', scale=[4.0, 4.0, 4.0]),
│   │   │   │   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   │   │   │   )
│   │   │   │   )
│   │   │   ),
│   │   │   coordinateTransformations=(VectorScale(type='scale', scale=[10.0, 10.0, 10.0]),),
│   │   │   metadata=None,
│   │   │   name='mouse tibia',
│   │   │   type=None,
│   │   │   version='0.4'
│   │   )
│   ],
│   omero=None
)

(
│   Dataset(
│   │   path='level0',
│   │   coordinateTransformations=(
│   │   │   VectorScale(type='scale', scale=[1.0, 1.0, 1.0]),
│   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   )
│   ),
│   Dataset(
│   │   path='level1',
│   │   coordinateTransformations=(
│   │   │   VectorScale(type='scale', scale=[2.0, 2.0, 2.0]),
│   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   )
│   ),
│   Dataset(
│   │   path='level2',
│   │   coordinateTransformations=(
│   │   │   VectorScale(type='scale', scale=[4.0, 4.0, 4.0]),
│   │   │   VectorTranslation(type='translation', translation=[0.0, 0.0, 0.0])
│   │   )
│   )
)

<zarr.core.Array '/level0' (1672, 349, 414) uint8>

iterate chunk-wise


In [3]:
zarr_arr = zarr_group[metadata.multiscales[0].datasets[0].path]
chunk_shape = zarr_arr.chunks
shape = zarr_arr.shape

colour=1
for i in range(0, shape[0], chunk_shape[0]):
    for j in range(0, shape[1], chunk_shape[1]):
        for k in range(0, shape[2], chunk_shape[2]):
            chunk_idx = (
                i // chunk_shape[0] * (shape[1] // chunk_shape[1]) * (shape[2] // chunk_shape[2]) +
                j // chunk_shape[1] * (shape[2] // chunk_shape[2]) +
                k // chunk_shape[2]
            )
            if True or chunk_idx % 2 == 1:
                chunk = zarr_arr[
                    i:i+chunk_shape[0],
                    j:j+chunk_shape[1],
                    k:k+chunk_shape[2]
                ]
                chunk = (chunk > 50) * colour
                zarr_arr[
                    i:i+chunk_shape[0],
                    j:j+chunk_shape[1],
                    k:k+chunk_shape[2]    
                ] = chunk
                colour = (colour+1)%256
                if colour == 0:
                    colour = colour+1
                print(f"Thresholded chunk at ({i}, {j}, {k})")
                print(f"colour: {colour}")

for d in [0,1,2]:
    level_d_array = zarr_group[ome_zarr_image.attributes.multiscales[0].datasets[d].path]
    level_d_array[:] = zarr_arr[::2**d, ::2**d, ::2**d]

Thresholded chunk at (0, 0, 0)
colour: 2
Thresholded chunk at (0, 0, 100)
colour: 3
Thresholded chunk at (0, 0, 200)
colour: 4
Thresholded chunk at (0, 0, 300)
colour: 5
Thresholded chunk at (0, 0, 400)
colour: 6
Thresholded chunk at (0, 100, 0)
colour: 7
Thresholded chunk at (0, 100, 100)
colour: 8
Thresholded chunk at (0, 100, 200)
colour: 9
Thresholded chunk at (0, 100, 300)
colour: 10
Thresholded chunk at (0, 100, 400)
colour: 11
Thresholded chunk at (0, 200, 0)
colour: 12
Thresholded chunk at (0, 200, 100)
colour: 13
Thresholded chunk at (0, 200, 200)
colour: 14
Thresholded chunk at (0, 200, 300)
colour: 15
Thresholded chunk at (0, 200, 400)
colour: 16
Thresholded chunk at (0, 300, 0)
colour: 17
Thresholded chunk at (0, 300, 100)
colour: 18
Thresholded chunk at (0, 300, 200)
colour: 19
Thresholded chunk at (0, 300, 300)
colour: 20
Thresholded chunk at (0, 300, 400)
colour: 21
Thresholded chunk at (100, 0, 0)
colour: 22
Thresholded chunk at (100, 0, 100)
colour: 23
Thresholded chun

In [4]:
import napari

viewer = napari.Viewer()
viewer.open(path, plugin="napari-ome-zarr")
napari.run()

Failed to parse metadata
Traceback (most recent call last):
  File "/home/alessandro/mambaforge/envs/big-imaging-data-tutorial/lib/python3.13/site-packages/ome_zarr/reader.py", line 340, in __init__
    rdefs = self.image_data.get("rdefs", {})
            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


/home/alessandro/mambaforge/envs/big-imaging-data-tutorial/lib/python3.13/site-packages/napari/layers/_layer_actions.py:83: UserWarning: projection mode "none" is not compatible with Labels layers. Falling back to "none".
  warnings.warn(


Right-click on the layer, and hit convert to Labels.